In [1]:
import panel as pn
import os
import numpy as np
import bokeh

from bokeh.layouts import column, row, Spacer
from bokeh.models import CustomJS, Slider, RadioButtonGroup, TextInput, Button, MultiChoice
from bokeh.models import BoxAnnotation, PreText, Range1d, LinearAxis, Span, HoverTool, DataTable, TableColumn
from bokeh.events import SelectionGeometry
from bokeh.plotting import ColumnDataSource, figure, show

import sys
import uuid
import logging

pn.extension('terminal')

In [2]:
import sys
sys.path.append('/home/jhoskins/anaconda3/lib/python3.8/site-packages')

In [3]:
print(sys.path)

['/home/jhoskins/Development/benchmarking', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/jhoskins/.local/lib/python3.9/site-packages', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.9/dist-packages', '/home/jhoskins/.local/lib/python3.9/site-packages/IPython/extensions', '/home/jhoskins/.ipython', '/home/jhoskins/anaconda3/lib/python3.8/site-packages']


In [4]:
#!/home/jhoskins/anaconda3/bin/python -m pip install casatasks==6.2.0.124
#!/home/jhoskins/anaconda3/bin/python -m pip install casatools==6.2.0.124
#!/home/jhoskins/anaconda3/bin/python -m pip install casatestutils==6.2.0.124

In [5]:
#import casatasks
#import casatools

In [6]:
import yaml
import param

class TCleanOptionsBaseClass(param.Parameterized):
    vis = param.String(default="E2E6.1.00034.S_tclean.ms", doc="Measurement file.") 
    imagename = param.String(default="standard_cube", doc="Name stub of output file.") 
    imsize = param.ListSelector(default=[80, 80])
    cell = param.String(default="1.1arcsec", doc="Cell size") 
    specmode = param.String(default="cube", doc="Specmode") 
    interpolation = param.String(default="nearest", doc="") 
    nchan = param.Integer(508, bounds=(1, 5000))
    start = param.String(default="220.2526743594GHz", doc="") 
    width = param.String(default="0.2441741MHz", doc="") 
    pblimit = param.Number(0.2)
    deconvolver = param.String(default="hogbom", doc="") 
    niter = param.Integer(1, bounds=(0, None))
    cyclefactor = param.Integer(2, bounds=(1, 50))
    scales = param.ListSelector(default=[0, 3, 10], objects=[0, 3, 5, 7, 9, 10], precedence=0.5)
    interactive = param.Integer(0, doc="Interactive mode")
    field = param.String(default='1')
    spw = param.ListSelector(default=['0'])
    antenna = param.ListSelector(default=['0,1,2,3,4,5,6,7,8'])
    scan = param.ListSelector(default=['8,12,16'])
    intent = param.String(default='OBSERVE_TARGET#ON_SOURCE')
    datacolumn = param.String(default='data')
    phasecenter = param.String(default='ICRS 00:45:54.3836 -073.15.29.413')
    stokes= param.String(default='I')
    outframe = param.String(default='LSRK') 
    perchanweightdensity= param.Boolean(False)
    gridder = param.String(default='standard')  
    mosweight = param.Boolean(False)
    usepointing = param.Boolean(False) 
    restoration = param.Boolean(False)
    pbcor = param.Boolean(False) 
    weighting = param.String(default='briggs') 
    restoringbeam = param.String('common')
    robust = param.Number(default=0.5) 
    npixels = param.Integer(default=0) 
    threshold = param.String(default='0.0mJy')
    nsigma = param.Number(default=0.0)
    usemask = param.String(default='auto-multithresh')
    sidelobethreshold = param.Number(1.25)
    noisethreshold = param.Number(5.0)
    lownoisethreshold = param.Number(2.0)
    negativethreshold = param.Number(0.0) 
    minbeamfrac = param.Number(0.1)
    growiterations = param.Integer(75)
    dogrowprune = param.Boolean(True)
    minpercentchange = param.Number(1.0)
    fastnoise = param.Boolean(False)
    savemodel = param.String(default='none')
    parallel = param.Boolean(False)
    verbose = param.Boolean(True)
    restart = param.Boolean(False)
    calcres = param.Boolean(True)
    calcpsf = param.Boolean(True)
    
    def read_configuration(self, config:str)->None:
        with open(config) as file:
            config = yaml.full_load(file)
        
            self.niter = config['tclean-second-cycle']['niter']
            self.pbcor = config['tclean-second-cycle']['pbcor']
            self.restart = config['tclean-second-cycle']['restart']
            self.calcres = config['tclean-second-cycle']['calcres']
            self.calcpsf = config['tclean-second-cycle']['calcpsf']
            self.threshold = config['tclean-second-cycle']['threshold']
            self.restoration = config['tclean-second-cycle']['restoration']

In [7]:
#options = TCleanOptions()
#options.read_configuration('stakeholder_tools/config/tclean.yaml')

In [11]:
class TCleanPanel(TCleanOptionsBaseClass):
    
    def __init__(self, terminal=False):
        self.terminal = terminal
        
        self.read_configuration('stakeholder_tools/config/tclean.yaml')
        
        self.file_widget = pn.widgets.FileSelector(os.getcwd(), name="File Selector")
        
        self.file_widget.param.watch(self.update, 'value')
        
        imsize_widget = pn.Param(
            TCleanOptionsBaseClass.param.imsize, 
            widgets={
                'imsize': pn.widgets.LiteralInput
            })
        
        cell_widget = pn.Param(
            TCleanOptionsBaseClass.param.cell, 
            widgets={
                'cell': pn.widgets.TextInput
            })
        
        specmode_widget = pn.Param(
            TCleanOptionsBaseClass.param.specmode, 
            widgets={
                'specmode': pn.widgets.TextInput
            })
        
        nchan_widget = pn.Param(
            TCleanOptionsBaseClass.param.nchan, 
            widgets={
                'nchan': pn.widgets.IntSlider
            })
        
        interactive_widget = pn.Param(
            TCleanOptionsBaseClass.param.interactive, 
            widgets={
                'interactive': pn.widgets.IntInput
            })
        
        # ------------------------------------ #
    
        self.play_button = pn.widgets.Button(
            name="Play", 
            button_type="success",  
            margin=(5,1,5,1)) 
    
    
        # ------------------------------------ #
    
        self.terminal_widget = pn.widgets.Terminal(
            "CASA TClean Terminal Experience\n\n",
            options={"cursorBlink": True},
            height=300, sizing_mode='stretch_width',
            name='Terminal'
        )
    
    
        simple_controls = pn.Column(
            imsize_widget,
            cell_widget,
            specmode_widget,
            nchan_widget,
            interactive_widget,
            self.play_button
        )
        
        advanced_controls = pn.Column(
            self.play_button)
        
        self.layout = pn.Row(
            pn.Column(
                pn.Accordion(self.file_widget),
                pn.Accordion( 
                    ("TClean", pn.Tabs(('Simple', simple_controls), ('Advanced', advanced_controls)))
                ),
                pn.Accordion(self.terminal_widget),
                width=500
            )
        )
    
        if self.terminal is True:
            self.layout.show()
    
    def update(self, event):
            self.vis = self.file_widget.value[0]
        

In [12]:
tclean = TCleanPanel()

In [13]:
tclean.layout

Row
    [0] Column(width=500)
        [0] Accordion
            [0] FileSelector(name='File Selector')
        [1] Accordion
            [0] Tabs
                [0] Column
                    [0] Param(ParameterizedMetaclass, name='TCleanOptionsBaseClass', parameters=['imsize'], show_name=False, widgets={'imsize': <class '...})
                    [1] Param(ParameterizedMetaclass, name='TCleanOptionsBaseClass', parameters=['cell'], show_name=False, widgets={'cell': <class '...})
                    [2] Param(ParameterizedMetaclass, name='TCleanOptionsBaseClass', parameters=['specmode'], show_name=False, widgets={'specmode': <...})
                    [3] Param(ParameterizedMetaclass, name='TCleanOptionsBaseClass', parameters=['nchan'], show_name=False, widgets={'nchan': <class '...})
                    [4] Param(ParameterizedMetaclass, name='TCleanOptionsBaseClass', parameters=['interactive'], show_name=False, widgets={'interactive': <...})
                    [5] Button(button_type='success', margin=(5, 1, 5, 1), name='Play')
                [1] Column
                    [0] Button(button_type='success', margin=(5, 1, 5, 1), name='Play')
        [2] Accordion
            [0] Terminal(id=139978563517840)